In [2]:
import pandas as pd
from mysql import connector
import pandas as pd

csv_files = {
    "ap_bus_details.csv": "Andhra Pradesh",
    "assam_bus_details.csv": "Assam",
    "himachal_bus_details.csv": "Himachal Pradesh",
    "kaac_bus_details.csv": "Karnataka",
    "kerala_bus_details.csv": "Kerala",
    "rajasthan_bus_details.csv": "Rajasthan",
    "sb_bus_details.csv": "Southbenal Buses",
    "Telangana_bus_details.csv": "Telangana",
    "up_bus_details.csv": "Uttar Pradesh",
    "wb_bus_details.csv": "West Bengal",
    "wb2_bus_details.csv": "West Bengal 2",
    "kadamba_bus_details.csv":"Kadamba"
}

dataframes = []

for file, source in csv_files.items():
    try:
        # Try reading the file and add the 'Source' column
        df = pd.read_csv(file)
        if df.empty:
            print(f"Skipping empty file: {file}")
            continue
        df['Source'] = source
        dataframes.append(df)
    except pd.errors.EmptyDataError:
        print(f"Skipping empty or invalid file: {file}")
    except Exception as e:
        print(f"Error reading file {file}: {e}")

# Combine all valid DataFrames
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Combined DataFrame created successfully.")
    # Save combined DataFrame for further use
    combined_df.to_csv("bus_routes.csv", index=False)
else:
    print("No valid data to combine.")

# Connect to MySQL
myconnection = connector.connect(
    host='127.0.0.1', user='root', passwd='123456789', database="redbus"
)

cursor = myconnection.cursor()

# Prepare SQL table schema
column_definitions = ", ".join(
    f"`{col}` {'FLOAT' if dtype == 'float64' else 'TEXT' if dtype == 'object' else 'INT'}"
    for col, dtype in zip(combined_df.columns, combined_df.dtypes)
)

# Create table
table_name = "bus_routes"
cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
cursor.execute(f"CREATE TABLE {table_name} ({column_definitions});")

# Prepare parameterized query for safe insertion
columns = ', '.join(f'`{col}`' for col in combined_df.columns)
placeholders = ', '.join(['%s'] * len(combined_df.columns))
query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert data into the table
for _, row in combined_df.iterrows():
    cursor.execute(query, tuple(row))

# Commit changes and close the connection
myconnection.commit()
cursor.close()
myconnection.close()

print("Data successfully inserted into MySQL database.")


Skipping empty or invalid file: kaac_bus_details.csv
Combined DataFrame created successfully.
Data successfully inserted into MySQL database.
